In [ ]:
import pandas
import re
import datetime
import glob

from dateutil.parser import parse as dateutil_parse
from mma.glm.merge_data import DATA_PATH, load_fighters_df, load_merged_df

pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', None)

In [ ]:
DUPED_IDS = {
    '0778f94eb5d588a5',
    '12ebd7d157e91701',
    '159da492e000afbf',
    '2924886ee9c4c527',
    '3dc3022232b79c7a',
    '7bc475b0fc2020ab',
    '8349d55cdecd393a',
    '8d9586726252dbf7',
    '99bcdf5eac39898f',
    'ac5a7400da3a9a41',
    'b2f88c04c4dd43ce',
    'd3de4a24f7eefac8',
    'dea070ed4a2a8281',
    'dfae5da5676e4194',
    'e0b74df14f52cd15',
    'eee0ef3e2b14816b',
    'f2477cf43c4975cc',
    'fb3e61720be4690c',
    'fdfef29ba17ee525'
}

MANUAL_MAPPING = pandas.DataFrame([
    {'fighterId': 'c739c2995a275314', 'espnId': '4917145'},
    {'fighterId': '82529ce93cd9a2cf', 'espnId': '4875513'},
    {'fighterId': '3f11fd1751fa83b1', 'espnId': '4894925'},
    {'fighterId': 'd8c811df0386d5e8', 'espnId': '5068624'},
    {'fighterId': '148bb103cfbf123e', 'espnId': '5068617'},
    {'fighterId': 'b671bdf981ad527d', 'espnId': '5068545'},
    {'fighterId': '036e96c1c12b8a59', 'espnId': '5122794'},
    {'fighterId': '082eba4cd80f736f', 'espnId': '5093447'},
    {'fighterId': '23a6e307077c6ccc', 'espnId': '3951294'},
    {'fighterId': '7b39035fae7268b8', 'espnId': '2983499'},
    {'fighterId': '1abfb658cd4f8533', 'espnId': '3012403'},
    {'fighterId': '49e49b54e5901d0d', 'espnId': '2612668'},
    {'fighterId': '668222c99c5c311d', 'espnId': '2558192'},
    {'fighterId': '338a11d3674eb2d4', 'espnId': '2536134'},
    {'fighterId': 'c058823a2595ab09', 'espnId': '2558133'},
    {'fighterId': '5898357a45a73674', 'espnId': '2558132'},
    {'fighterId': 'e8efeb9cf33b1941', 'espnId': '2951312'},
    {'fighterId': 'c6e6926a81adcd00', 'espnId': '2558141'},
    {'fighterId': '7ca4c3f8aa8bacae', 'espnId': '2558143'},
    {'fighterId': 'e8c170a64dc920ac', 'espnId': '2558140'},
    {'fighterId': '56f4b81ec4db61af', 'espnId': '2951489'},
    {'fighterId': '21f2974fd08085e3', 'espnId': '2951254'},
    {'fighterId': '53e533db1b8e9712', 'espnId': '2951342'},
    {'fighterId': 'abbc4fc02e0d84b3', 'espnId': '2951316'},
    {'fighterId': '4985113c0928aa62', 'espnId': '2951291'}
#     {'fighterId': '', 'espnId': ''},
#     {'fighterId': '', 'espnId': ''},
#     {'fighterId': '', 'espnId': ''},
])

In [ ]:
def name_to_slug(name: str) -> str:
    return '-'.join([re.sub(r'\W+', '', word) for word in name.split()]).lower()

In [ ]:
all_espn_fighters = pandas.read_csv(f'{DATA_PATH}/espn_fighters.csv')
all_espn_fighters['espnId'] = all_espn_fighters['espnId'].astype(str)

In [ ]:
all_ufc_fighters = load_fighters_df()
all_ufc_fighters['ufcSlug'] = all_ufc_fighters['FIGHTER'].apply(name_to_slug)
all_ufc_fighters.head(1)

In [ ]:
duped_merged_df = load_merged_df()[['fighterId','opponentId','date']]
duped_merged_df['count'] = 1

ufc_counts = duped_merged_df.groupby(['fighterId','date']).aggregate({'count': 'count'})
ufc_df = ufc_counts[ufc_counts['count'] == 1].merge(duped_merged_df, on=['fighterId','date'], how='left')

In [ ]:
espn_fight_dfs = []
for filename in glob.glob(f'{DATA_PATH}/espn_fighters/*/distance.csv'):
    espn_fight_dfs.append(pandas.read_csv(filename))
espn_fights = pandas.concat(espn_fight_dfs, axis=0)
espn_fights['espnId'] = espn_fights['espnId'].astype(str)
espn_fights['opponentEspnId'] = espn_fights['opponentEspnId'].astype(str)
espn_fights['date'] = espn_fights['Date'].apply(dateutil_parse)

# sometimes fighters have fought multiple times per day
duped_espn_df = espn_fights[espn_fights['date'] >= min_ufc_date][['espnId', 'opponentEspnId', 'date']].dropna().reset_index(drop=True)
duped_espn_df['count'] = 1

In [ ]:
espn_counts_df = duped_espn_df.groupby(['espnId','date'], as_index=False).aggregate({'count': 'count'})
espn_df = (
    espn_counts_df[espn_counts_df['count'] == 1]
    .drop(columns=['count'])
    .merge(duped_espn_df[['espnId', 'date','opponentEspnId']], on=['espnId', 'date'], how='left')
)

In [ ]:
espn_fighters = (
    espn_df[['espnId']].groupby(['espnId'],as_index=False).nth(0).reset_index(drop=True)
    .merge(all_espn_fighters, on='espnId', how='inner')
)
espn_fighters.head()

In [ ]:
ufc_fighters = (
    ufc_df[['fighterId']].groupby(['fighterId'],as_index=False).nth(0).reset_index(drop=True)
    .merge(all_ufc_fighters, on='fighterId', how='inner')
)
ufc_fighters.head()

In [ ]:
ufc_espn = ufc_fighters.merge(espn_fighters, left_on='ufcSlug', right_on='espnSlug', how='left').dropna()
espn_ufc = ufc_fighters.merge(espn_fighters, left_on='ufcSlug', right_on='espnSlug', how='right').dropna()

duped_mapping = pandas.concat([ufc_espn, espn_ufc], axis=0)[['fighterId', 'espnId']]
init_mapping = (
    pandas.concat([duped_mapping[~duped_mapping['fighterId'].isin(DUPED_IDS)], MANUAL_MAPPING], axis=0)
    .drop_duplicates(subset=['fighterId'], ignore_index=True)
    .drop_duplicates(subset=['espnId'], ignore_index=True)
    .reset_index(drop=True)
)

all_ids = set(ufc_fighters['fighterId'])
mapped_ids = set(init_mapping['fighterId'])

print(f'{len(mapped_ids):,} / {len(all_ids):,} ({len(mapped_ids)/len(all_ids):.1%}) of UFC ids mapped')

In [ ]:
"""
mapping algo...
- init maps we know for sure
- make DF of UFC fighter1, fighter2, date
- plus flipping 1 <=> 2

- make DF of ESPN fighter1, fighter2, date plus flipping
- merge in known mapping
- induce missing maps from fights
- set init map = induced map
- run until we 
""";

In [ ]:
mapping = init_mapping[['fighterId','espnId']].copy()

old_n_mapping = 0
new_n_mapping = len(mapping)
print(f'initial mapping: {new_n_mapping}')

In [ ]:
# ufc_df[ufc_df['fighterId'] == '1dab0d1d81dd06db']

In [ ]:
max_runs = 100
n_runs = 0
while old_n_mapping < new_n_mapping and n_runs < max_runs:
    mapped_ufc_df = ufc_df.merge(mapping, on='fighterId', how='inner').dropna()
    merged_df = mapped_ufc_df.merge(espn_df, on=['espnId', 'date'], how='inner').dropna()
  
    # if we have mapped fighter UFC <=> ESPN,
    # then we can map their opponents
    # via matching up fight dates
    add_to_mapping = (
        merged_df[['opponentId','opponentEspnId']]
        .rename(columns={'opponentId': 'fighterId', 'opponentEspnId': 'espnId'})
        .drop_duplicates(ignore_index=True)
    )

    mapping = pandas.concat([mapping, add_to_mapping], axis=0).drop_duplicates(ignore_index=True)
    old_n_mapping = new_n_mapping
    new_n_mapping = len(mapping)
    print(f'{old_n_mapping=}, {new_n_mapping=}')
    n_runs += 1

In [ ]:
mapping['count'] = 1
dupes = mapping.groupby(['fighterId'], as_index=False).aggregate({'count': 'count'})
duped_ids = set(dupes[dupes['count'] > 1]['fighterId'])
dupes[dupes['count'] > 1]

In [ ]:
DATA_PATH

In [ ]:
mapping.drop(columns=['count']).to_csv(f'{DATA_PATH}/ufcstats_espn_mapping.csv', index=False)

In [ ]:
last_fights = duped_merged_df.groupby(['fighterId'], as_index=False).aggregate({'date': 'max'})
missing_maps = last_fights.merge(ufc_fighters[~ufc_fighters['fighterId'].isin(mapping['fighterId'])], how='right').sort_values(['date'], ascending=False)
print(f'missing {len(missing_maps)} maps')
missing_maps.head(10)

In [ ]:
all_espn_fighters.head()

In [ ]:
new_manual = missing_maps.merge(espn_fighters, left_on=['ufcSlug'], right_on=['espnSlug'], how='inner')

In [ ]:
mapping[mapping['fighterId'].isin(new_manual['fighterId'])]

In [ ]:
mapping[mapping['espnId'].isin(new_manual['espnId'])]

In [ ]:
new_manual[['fighterId', 'espnId']].to_dict(orient='records')

In [ ]:
# mapping[mapping['espnId'] == '4917145']